In [1]:
import numpy as np
import joblib
import requests
from pymystem3 import Mystem
import re
from nltk.corpus import stopwords
from sklearn.metrics import precision_score , recall_score

In [2]:
bigdata = []

## Выгрузка данных из 3ех новостных групп в вконтакте

 За тематику текстов для классификации выбрал следущее:
 Новости про covid, Спам в новостях, Непосредственно новости.

 Сами данные я решил сам разметить, а не выбирая сразу данные из конкретных тематик,
 например из отдельных групп с новостями про : науку, экономику, здоровье, животных и т.д.

 Я выбрал группы, которые выкладывают новости об всем что угодно, и пытаюсь сам их разметить.

In [1]:
#https://oauth.vk.com/authorize?client_id=7990988&display=page&scope=wall&response_type=token&v=5.131

In [3]:
token ='34e0cbf531055e09361376487d0bd9704478ca1072a730091388deaf4a62c22e8f2003c8e47b7cf6d481a'

In [4]:
# https://vk.com/toporcc
# https://vk.com/plohie_novosti_mc
# https://vk.com/lentach
IDS = 'toporcc','plohie_novosti_mc','lentach'

for ID in IDS:
    json_response = requests.get(('https://api.vk.com/method/wall.get?domain={}&count=100&v=5.103&access_token=' + token).\
                                 format(ID)).json()
    if json_response.get('error'):
        print(json_response.get('error'))
    else:
        for item in json_response['response']['items']:
            bigdata.append(item['text'])

In [5]:
len(bigdata)

300

## Разметка выгруженных данных

 Данный способ имеет большую погрешность в разметке, потому, что 'ключевые слова' из различных классов могут быть употреблены
 
 В другом классе, но при этом этот алгоритм разметки отнесет его в класс по 'ключевому слову'
 
 Лучше смотреть на смысловую часть предложений\кусков текста используя готовые методы из NLP

In [7]:
spamwords = ['кредит','заем','займ','подарить','промокод','активация','выгода','халявный','резюме','smartzaem']

In [8]:
covidwords = ['коронавирус','вирус','заболеть','болеть','больной','заражать','пандемия','coronavirus','вакцинация']

In [9]:
m = Mystem()

In [10]:
def lemmatize_sentence(text):
    lemmas=m.lemmatize(text)
    return "".join(lemmas).strip()

In [11]:
spam = []
covid = []
news = []

testset = []
dataset = []

In [12]:
# ~20% из данных будут тестом

# Русские стоп слова, их нужно отбросить (союзы,предлоги)
stop_words = stopwords.words("russian")
    
for data in bigdata:
    
    if np.random.rand(1)>0.2:
        
        # Приведение слов к их инфинитиву
        doc = lemmatize_sentence(data)
        # Разбиение на слова предложения
        tokens_data=re.split(r'[-\s\n\t-:\[\]<>\'.,;!?()" ]+',doc)
        
        words_data = []
        
        # Получение выборки
        for token_data in tokens_data:
            if ((token_data not in stop_words) and (len(token_data)>3)):
                
                words_data.append(token_data)
                
                
        # Разметка данных
        c=0
        
        for spamword in spamwords:
            if spamword in words_data:
                spam.append(words_data)
                #dataset.append([words_data,0])
                c+=1
                break
            
        if c==0:
            for covidword in covidwords:
                if covidword in words_data:
                    covid.append(words_data)
                    #dataset.append([words_data,1])
                    c+=1
                    break
    
        if c==0:
            news.append(words_data)
            #dataset.append([words_data,2])
            
    else:

        # Приведение слов к их инфинитиву
        doc = lemmatize_sentence(data)
        # Разбиение на слова предложения
        tokens_data=re.split(r'[-\s\n\t-:\[\]<>\'.,;!?()" ]+',doc)
        
        # Получение выборки 
        words_data = []
        
        for token_data in tokens_data:
            if ((token_data not in stop_words) and (len(token_data)>3)):
                words_data.append(token_data)
        
        # Заполнение тестового датасета
        testset.append(words_data)

In [18]:
dataset = []

In [19]:
def upd_dataset(array,num_class):
    for data in array:
        dataset.append([data,num_class])

In [20]:
upd_dataset(spam,0)
upd_dataset(covid,1)
upd_dataset(news,2)

In [39]:
# Объем тестовой выборки
len(testset)

68

In [41]:
# Объем тренировочной выборки
len(dataset)

232

## Обучение (Наивный байесовский классификатор)

In [23]:
def fit (dataset, alpha):
    classes, freq, tot_in_cl, total = {}, {}, {}, set()
    for features, label in dataset:
        if label not in classes:
            classes[label] = 0
            tot_in_cl[label] = 0
        classes[label] += 1
        for feature in features:
            if (feature,label) not in freq:
                freq[(feature,label)] = 0
            freq[(feature,label)] += 1
            tot_in_cl[label] += 1
            total.add(feature)
            
            
    # Перевод на язык вероятностей
    # Частоты слов
    for feature,label in freq:
        freq[(feature,label)] = ((alpha + freq[(feature,label)] ) /  (alpha * len(total) + tot_in_cl[label]) )
        
    # Частоты классов    
    for cl in classes:
        classes[cl] /= len(dataset)
        
    
    return alpha,classes, freq, tot_in_cl, len(total)

def predict(classifier , features):
    alpha, classes, freq, tot_in_cl, len_total = classifier
    # Лямбда функция
    return max(classes.keys(),
              key = lambda cl:
              np.log10(classes[cl]) + 
                sum(np.log10(freq.get((feature,cl), alpha/(alpha*len_total+tot_in_cl[cl])) )  \
                  for feature in features ))

In [24]:
model = fit(dataset,1)

In [25]:
# Сохранение обученной модели
#joblib.dump(model,'model_dict.pkl')

# Использование обученной модели
#model = joblib.load('model_dict.pkl')

['model_dict.pkl']

## Разметка тестовых данных (y) ,  и прогнозирование модели (y_pred)

In [26]:
y = []

In [27]:
for test in testset:
    c = 0
    for spamword in spamwords:
        if spamword in test:
            y.append(0)
            c+=1
            break
            
    if c==0:
        for covidword in covidwords:
            if covidword in test:
                y.append(1)
                c+=1
                break
    
    if c==0:
        y.append(2)

In [28]:
y_pred = []

In [29]:
for testdata in testset:
    y_pred.append(predict(model,testdata))

# Подсчет F меры, так как классы не сбалансированы

In [30]:
pr_score = precision_score(y,y_pred,average = 'micro')

In [31]:
rec_score = recall_score(y, y_pred,average = 'micro')

In [32]:
F = pr_score*rec_score/(pr_score+rec_score)

In [33]:
F

0.3382352941176471